In [ ]:
!pip install mixpanel-utils 

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

import os
from datetime import datetime
import pymongo
import json
import tempfile
import uuid
from tqdm.auto import tqdm
from mixpanel_utils import MixpanelUtils

from bot import config
from bot import database

In [ ]:
def split_into_chunks(seq, chunk_size):
    for i in range(0, len(seq), chunk_size):
        yield seq[i:i + chunk_size]

In [ ]:
db = database.Database()

api_secret = ""
project_token = ""
mputils = MixpanelUtils(
    api_secret,
    project_token,
)

In [ ]:
# # just to save commands
# mputils.export_people("tmp.json", {'from_date':'2023-03-01','to_date':'2023-03-27','event': ["start"]})
# mputils.export_people("tmp.json", format='json')

### people_set

In [ ]:
users_to_dump = []
query = db.user_collection.find({})
n = db.user_collection.count_documents({})

for user_dict in tqdm(query, total=n):        
    properties = user_dict
    properties["first_seen"] = properties["first_seen"].timestamp()
    properties["last_interaction"] = properties["last_interaction"].timestamp()
    
    users_to_dump.append({
        "$distinct_id": user_dict["_id"],
        "$properties": user_dict
    })

for chunk in tqdm(split_into_chunks(users_to_dump, 500)):
    with tempfile.NamedTemporaryFile() as tmp_file:
        with open(tmp_file.name, "w") as f:
            json.dump(chunk, f)

        mputils.import_people(tmp_file.name)

### start

In [ ]:
event_name = "start"

events_to_dump = []
query = db.user_collection.find({})
n = db.user_collection.count_documents({})
for user_dict in tqdm(query, total=n):    
    distinct_id, event_name, properties = (
        user_dict["_id"],
        event_name,
        {}
    )
    properties["time"] = user_dict["first_seen"].timestamp()
    properties["distinct_id"] = str(distinct_id)
    properties["$insert_id"] = str(uuid.uuid4())
    
    events_to_dump.append({
        "event": event_name,
        "properties": properties,
    })

    
for chunk in tqdm(split_into_chunks(events_to_dump, 500)):
    with tempfile.NamedTemporaryFile() as tmp_file:
        with open(tmp_file.name, "w") as f:
            json.dump(chunk, f)

        mputils.import_events(tmp_file.name, timezone_offset=0)

### successful_payment

In [ ]:
event_name = "successful_payment"

events_to_dump = []
query = db.payment_collection.find({"status": "paid"})
n = db.payment_collection.count_documents({"status": "paid"})
for payment_dict in tqdm(query, total=n):        
    payment_method = payment_dict.get("payment_method", "cryptomus")
    product = payment_dict.get("product", "default")
    
    amount = payment_dict["amount"]
    currency = payment_dict.get("currency", "USD")
    if currency == "RUB":
        amount /= 77
    
    distinct_id, event_name, properties = (
        payment_dict["user_id"],
        event_name,
        {
            "payment_method": payment_method,
            "product": product,
            "payment_id": payment_dict["_id"],
            "amount": amount
        }
    )
    properties["time"] = payment_dict["created_at"].timestamp()
    properties["distinct_id"] = str(distinct_id)
    properties["$insert_id"] = str(uuid.uuid4())
    
    events_to_dump.append({
        "event": event_name,
        "properties": properties,
    })
    

for chunk in tqdm(split_into_chunks(events_to_dump, 500)):
    with tempfile.NamedTemporaryFile() as tmp_file:
        with open(tmp_file.name, "w") as f:
            json.dump(chunk, f)

        mputils.import_events(tmp_file.name, timezone_offset=0)

### send_message

In [ ]:
event_name = "send_message"

events_to_dump = []
query = db.dialog_collection.find({})
n = db.dialog_collection.count_documents({})
for dialog_dict in tqdm(query, total=n):
    for message in dialog_dict["messages"]:
        user_id = dialog_dict["user_id"]
        
        distinct_id, event_name, properties = (
            user_id,
            event_name,
            {
                "dialog_id": dialog_dict["_id"],
                "chat_mode": dialog_dict["chat_mode"]
            }
        )
        properties["time"] = message["date"].timestamp()
        properties["distinct_id"] = str(distinct_id)
        properties["$insert_id"] = str(uuid.uuid4())

        events_to_dump.append({
            "event": event_name,
            "properties": properties,
        })


for chunk in tqdm(split_into_chunks(events_to_dump, 500)):
    with tempfile.NamedTemporaryFile() as tmp_file:
        with open(tmp_file.name, "w") as f:
            json.dump(chunk, f)

        mputils.import_events(tmp_file.name, timezone_offset=0)

### send_invoice

In [ ]:
event_name = "send_invoice"

events_to_dump = []
query = db.payment_collection.find({"status": {"$ne": "paid"}})
n = db.payment_collection.count_documents({"status": {"$ne": "paid"}})
for payment_dict in tqdm(query, total=n):        
    payment_method = payment_dict.get("payment_method", "cryptomus")
    product = payment_dict.get("product", "default")
    
    amount = payment_dict["amount"]
    currency = payment_dict.get("currency", "USD")
    if currency == "RUB":
        amount /= 77
    
    distinct_id, event_name, properties = (
        payment_dict["user_id"],
        event_name,
        {
            "payment_method": payment_method,
            "product": product,
            "payment_id": payment_dict["_id"],
            "amount": amount
        }
    )
    properties["time"] = payment_dict["created_at"].timestamp()
    properties["distinct_id"] = str(distinct_id)
    properties["$insert_id"] = str(uuid.uuid4())
    
    events_to_dump.append({
        "event": event_name,
        "properties": properties,
    })
    

for chunk in tqdm(split_into_chunks(events_to_dump, 500)):
    with tempfile.NamedTemporaryFile() as tmp_file:
        with open(tmp_file.name, "w") as f:
            json.dump(chunk, f)

        mputils.import_events(tmp_file.name, timezone_offset=0)

---